# Intro to Programs and Measurements
----

This lesson is designed to introduce you to Pyquil's formalism for running quantum circuits, specifically creating quantum systems using programs and measurements.

This lesson is reccomended for first time users of Pyquil.  If you do not have Pyquil ready for use on your computer, please check out this installation guide:

link

---

If you wish to skip around this lesson for the examples of code, please run the following cell of code below to ensure all functions are imported properly:

In [90]:
from pyquil.quil import Program
from pyquil.gates import I, H
from pyquil.api import QVMConnection

qvm = QVMConnection()

## Creating Our First Quantum State
---

Pyquil is a python language that allows us to do is create algorithms that we can then run on a quantum computer.  These algorithms tell the quantum computer what kinds of quantum systems to create, and then manipulate them with gates.

Let's start with the simplest quantum system there is:

$$ |\Psi \rangle = | 0 \rangle $$

This is a quantum system of 1 qubit, in the state |$0\rangle$.  Not terribly exciting, but we have to start somewhere!  Consider this the "Hello World!" to programing with qubits.

Let's see the code that genertes this system, and then disect it's components:

In [11]:
from pyquil.quil import Program
from pyquil.api import QVMConnection

qvm = QVMConnection()
hello_qubit = Program()
print(qvm.wavefunction(hello_qubit))

(1+0j)|0>


Congrats, you've just created your first quantum system using Pyquil!

$*$ *crickets chirping* $*$

Okay, it's not a very exciting result, but there are already a lot of things going on in this code.  Starting with our imports:

---

from pyquil.quil import **Program**

from pyquil.api import **QVMConnection**

---

These imports are what allow us to create and see the quantum system we are working with.  The underlying quantum language behind Pyquil is Quil (Quantum Instruction Language), which Pyquil allows us to work with in a much more user-friendly syntax.

Program -- this is a class that can be thought of as our "instructions" for the quantum system.  We will $\textit{store}$ operations into programs, sort of like a list, and then call upon other functions to run these operations later.

QVMConnection -- this is a cloud-based virtual machine that will $\textit{run}$ our quantum algorithms.  It is a classical simulator, which means that it comes with a lot of handy features that are normally impossible on real quantum computers (QPUs - quantum processing units).

The goal of this lesson is to become familiar with some of the basics of programs and QVM, so don't worry if they don't maske sense just yet.

Now, let's see the remaining three lines of code:

---

qvm = QVMConnection()

hello_qubit = Program()

print(qvm.wavefunction(hello_qubit))

---

The first line of code stores the QVMConnection function as 'qvm'.  The second line of code creates a new program, with no instructions, and stores it to a variable we've called 'hello_qubit'.  And the last line of code uses a function called 'wavefunction', an extention of qvm, to create a string that shows our quantum state stored in hello_qubit.  This string is a user-friendly version of the quantum system, meant for viewing.

By default, when we create a program with no instructions, Pyquil creates the state |$0\rangle$.  Thus, when we use wavefunction() to look at the system, we get exactly the state $ |\Psi \rangle = | 0 \rangle $ we wanted.


## Let's Bump Up the Qubits
---

1 qubit is pretty exciting (I know), so let's try creating a state of multiple qubits (woah, don't get too crazy on me now).

In the previous example we created a system of a single qubit, which by deafult is started in the state |0$\rangle$.  This was done by simply calling the function Program() with no arguments.  While this is okay, it's better to be more precise when we initialize a quantum state.  Consider the following two ways of creating the same system:

In [92]:
from pyquil.gates import I

program1 = Program()
program2 = Program(I(0))

print('program1 wavefunction: ',qvm.wavefunction(program1))
print('program2 wavefunction: ',qvm.wavefunction(program2))

program1 wavefunction:  (1+0j)|0>
program2 wavefunction:  (1+0j)|0>


As we can see, both ways produce the same result, but the second program has the argument I(0).  This argument is actually a gate, the Identity operator, which we will discuss shortly.  But essentially the identity operator is a gate that leaves the state of a qubit unchanged.

Note: we are using python notation here, where all ordering starts from 0,1,2,3...  So the first qubit will always be 'qubit 0', the second qubit will be 'qubit 1', and so on.  Thus, I(0) is a gate operation on qubit 0.

To check if program2 is really any different from program1, Pyquil allows us to print programs to console:

In [33]:
program1 = Program()
program2 = Program(I(0))

print('program1 : ',program1)
print('program2 : ',program2)

program1 :  
program2 :  I 0



Sure enough, they both produce the same wavefunction in the end, but program2 definitely has an extra step: apply the identity gate to qubit 0.

So why would we prefer to include this extra operator if it doesn't do anything?  In short, it's to make our code easier to read and understand.  Program( ) with no arguments is a little... sloppy, but Program( I(0) ) is much more clear - put qubit 0 into the system and leave it in the state |$0\rangle$.

More importantly however, if we want to create systems with multiple qubits, we need a basic way of telling our program to initialize them.  The identity gate allows us to call as many qubits as we want, in the form: I(0),I(1),I(2),...I(n), which in turn creates a basic quantum state of all the qubits in the state |$000...0\rangle$

In [14]:
from pyquil.gates import I

three_qubits = Program(I(0),I(1),I(2))

print('wavefunction: ',qvm.wavefunction(three_qubits))

wavefunction:  (1+0j)|000>


This code creates a quantum system of three qubits, all in the state |$0\rangle$ (three times as exciting as before!).  Note, there actually is a limit to how many qubits we can create, dictated by Rigetti's QVM (I believe 26 is the max), but in principle we can consider that this coding syntax is limitless.

## Our First Mixed State
---

Now that we can create multiple qubits, we want to $\textit{do something}$ with them.  In quantum algorithms, that something is applying gates and making measurements.  We've already seen one gate to far, I -- the identity opertor, so let's add one more to our toolbox.

Pyquil comes with several pre-programed gates from the file pyquil.gates.  For a complete list and explination of all these gates - see the .ipynb lesson called "Pyquil Gates."  Here, we are only going to need the following gates:

In [19]:
from pyquil.gates import I, H

For this intro lesson we will only be using two gates, I and H, and we will breifly explain them here:

### I (Idenity Gate)

As we already saw, this gate acts on a single qubit, and leaves it's state unchanged.  The matrix for this gate is:

$$
\begin{bmatrix} 
1 & 0\\ 
0 & 1
\end{bmatrix}
$$

While perhaps uninteresting in itself, the I gate is still an essential component for algorithms.  We've alredy seen it used so far to initialize nice simple quantum states, and later we shall see it constantly be used in conjunction with other gates, for much more large scale operations.


### H (Hadamard Gate)

This gate is going to allow us to create our first mixed state.  In particular, the Hadamard gate takes a qubit and splits it into a 50/50 probability distribution between the states |$0\rangle$ and |$1\rangle$.  Mathematically, it looks like this:

$$ H |0\rangle = \frac{1}{\sqrt{2}} ( |0\rangle + |1\rangle ) $$
$$ H |1\rangle = \frac{1}{\sqrt{2}} ( |0\rangle - |1\rangle ) $$

which is accomplished by the follwing matrix:

$$
\begin{bmatrix} 
1 & 1\\ 
1 & -1
\end{bmatrix}
$$

Let's see an example:

In [17]:
first_mixed = Program(I(0),H(0))

print('wavefunction: ',qvm.wavefunction(first_mixed))

wavefunction:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>


Sure enough, our qubit is in a mixed state!  Our qubit has a 50% chance of being in the states |$0\rangle$ and |$1\rangle$ each.  

Note: The numbers attached to the states here are the system's ampltudes, not probabilities.  When working with quantum states, probabilities are always the $\textit{observables}$ that we see, but the ampltiudes are the inner workings that really mattter.  Here, each state has an amplitude of $\frac{1}{\sqrt{2}}$, which when squared, tells us that each state has a probability of $\frac{1}{2}$.

Let's try making a mixed state of 2 qubits:

In [18]:
two_mixed = Program(H(0),H(1))

print('wavefunction: ',qvm.wavefunction(two_mixed))

wavefunction:  (0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>


The wavefunction printed above shows an equal suposition of four states.  These four states come from the following mathematical state:

$$
H|0\rangle \otimes H|0\rangle
$$

which is the tensor product of two seperate quantum states.  A more common way of writing this is:


$$ (H|0\rangle)(H|0\rangle) $$

$$=\frac{1}{\sqrt{2}}(0\rangle + |1\rangle) \cdot \frac{1}{\sqrt{2}}(0\rangle + |1\rangle)$$

$$= \frac{1}{2}(|0\rangle + |1\rangle)(|0\rangle + |1\rangle)$$

$$=\frac{1}{2}(|0\rangle |0\rangle + |0\rangle |1\rangle + |1\rangle |0\rangle + |1\rangle |1\rangle)$$

which is typically written using the standard shorthand:

$$=\frac{1}{2} ( |00\rangle + |01\rangle + |10\rangle + |11\rangle )$$

And voila! We have our mixed state resulting from two qubits, each with a Hadamard gate applied to them.  Recall that a single H gate put our qubit into a 50/50 state between |$0\rangle$ and |$1\rangle$.  Now, having two qubits undergo this gate, both of them in this 50/50 state, we get a combined system where any of the four individual combinations has a 25% probability.

As a side note, you may have noticed I did something a little sly in the first line of code above:

---

two_mixed = Program(H(0),H(1))

---

I didn't initialize the qubits using the Identity gate first! (what a hypocrite, I'm so ashamed...)  Actually, this is perfectly acceptable code writing.  The only time it is neccessary to initialize a qubit with I() is when we want to specifically start it out in the state |$0\rangle$.  

If we alreayd know that we want a given qubit to start in some initial state, like $\frac{1}{\sqrt{2}}(0\rangle + |1\rangle)$ for example, than it is perfectly fine to do so.  At a quick glance, Program( H(0),H(1) ) is still pretty easy to read and understand -- I am creating a system of two qubits, and I am starting both of them off with a Hadamard gate.

Consider the following example where I would like only one of my qubits to start off in a superpostion:

In [94]:
one_mixed = Program(H(0),I(1))

print('wavefunction: ',qvm.wavefunction(one_mixed))

wavefunction:  (0.7071067812+0j)|00> + (0.7071067812+0j)|01>


As shown above, qubit 0 is initialized in a mixed state, while qubit 1 remains in the state |$0\rangle$.  If the ordering of the qubits looks weird to you (like it did to me at first), that's because Pyquil writes the states of the qubits 'backwards' from normal convention.  In the example above the first 0 in each state is actually qubit 1, while the second values, 0 and 1, are qubit 0.

If you still want to initialize qubits with I() before applying gates, like the example 'first_mixed', go for it!  Understanding code is only as easy as you make, so feel free to add steps for clarity.

## Making a Measurement
---

Now comes the hardest section of this intro lesson -- measuring the quantum states that we create.  To do this, Pyquil has a convenient way for us to measure and record the results of our quantum system, but the syntax is a little tricky at first.

Let's see an example in action and then backtrack to understand each component:

In [23]:
qubit0 = 0
perfect_coin = Program( H(qubit0) )
creg_index = 0

perfect_coin.measure(qubit0,creg_index)
cregs_to_check = [0]
trials = 5

print(qvm.run(perfect_coin, cregs_to_check, trials))

[[1], [0], [0], [0], [1]]


Woah buddy, lot to unpack here!  Let's start with the first three lines of code, since they will be the easiest to undstand:

---

qubit0 = 0

perfect_coin = Program( H(qubit0) )

creg_index = 0

---

In the first line, I am simply creating a normal python variable 'qubit0' and setting it equal to the integer 0.  Then, I am creating my program, just like we have done so far.  But instead of explicity writing Program( H(0) ), I am opting to write it as Program( H(qubit0) ) instead.  And lastly, I am again assigning a plain old python variable called 'creg_index' to the integer 0.

So far, we haven't done anything new.  With these three lines of code alone, the state of our system is simply:

$$ |\Psi \rangle = \frac{1}{\sqrt{2}}( |0\rangle + |1\rangle ) $$

stored in the variable 'perfect_coin'.  Just to verify this, let's check it out using qvm.wavefunction():

In [24]:
qubit0 = 0
perfect_coin = Program( H(qubit0) )
creg_index = 0

print('wavefunction: ',qvm.wavefunction(perfect_coin))

wavefunction:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>


That's all well and good, but why did I create the variable 'creg_index'?  'creg' is my shorthand version of 'classical register', which is where we are going to store the results of our measurements.  The classical register is just a string of bits, all 0's and 1's, with the intention that these bits will hold our measurement results.

Thus '$\textit{creg_index}$' is what I have chosen to name my variable here, which can be thought of as "the integer index location in the classical register that I intend to store $\textit{qubit0}$'s measurement result".  So later, after a successful measurment, we will go back and check the classical register and look for the measurement result to be store at 0. 

In general, this is the typical flow of information when making measurements in Pyquil.  We prepare a system $\rightarrow$ make a measurement $\rightarrow$ record the measurement result (either 0 or 1) in the classical register.

Next, let's take a look at the line of code that is $\textit{actually}$ doing the measurement:

---

perfect_coin.measure(qubit0,creg_index)

---

.measure() is a function that comes with the Program() class, so there was no need to import it directly.  It takes two arguments -- a qubit index to measure, and a classical register index for storing.  The second argument here is optional, as we are not forced to record a measurement result if we choose not to.  Thus, the following code is perfectly acceptable:

In [38]:
perfect_coin = Program( H(0) )

perfect_coin.measure(0)

print('wavefunction: ',qvm.wavefunction(perfect_coin))

wavefunction:  (1+0j)|0>


This code starts off with qubit 0 in an equal superposition of states |$0\rangle$ and |$1\rangle$, via the Hadamard gate, and then makes a measurement.  We can peak at the wavefunction after the measurement, seeing which state was measured, but this information isn't actually stored anywhere.  And remember, in a real quantum system, we don't have wavefunction() to cheat and look at the system for us.  So if we make a measurement and don't record the results, we better be sure we didn't need that information!

It is worth noting that .measure() is actually changing our program here.  To see this, let's try printing our program to console, before and after .measure():

In [37]:
perfect_coin = Program( H(0) )
print('before .measure(): ')
print(perfect_coin)

perfect_coin.measure(0)
print('after  .measure(): ')
print(perfect_coin)


before .measure(): 
H 0

after  .measure(): 
H 0
MEASURE 0



So actually, .measure() is just adding another entry to our list of instructions stored in our program.  So when do we $\textit{actually}$ make a measurement!? (growing impatient are we?)  Well, since we are just compiling a list of things to do in our program, the line of code that executes all the instructions is:

---

qvm.run(perfect_coin, cregs_to_check, trials)

---

.run() is the function that... $\textit{runs}$ our quantum algorithm (shocker, i know).  The first argument is the program to run, the second argument is the classical register locations to check afterwards, and the thrird argument is the number of times to repeat .run().  Thus, in order to perform the measurement and record it's result into the classical register, we must call this .run() function.

But wait one gosh darn minute! How come two examples ago we could see the wavefunction after .measure(), without usin .run()?  (Nothing gets by you, huh)  This is because wavefunction() is using the same classical simulator -- qvm.  If we look closely, both .run() and .wavefunction() are extensions from qvm.  Thus, .wavefunction() carries out any program, with all of it's instructions, and calculates the resulting wavefunction.  

BUT, the real difference between these two functions is that the program is not actually being executed when we use .wavefunction().  We are essentially $\textit{simulating}$ the program and seeing what the final state of the system $\textit{would}$ be.  .run() on the otherhand physically runs the program, which means any measurement results become $\textbf{permanent}$! (...dramatic much?)

If you don't totally get what's going on with QVM and the classical register, don't worry.  In future lessons we dive deeper into these topics.  For now, let's wrap up this section by returning to our complete example:

In [42]:
qubit0 = 0
perfect_coin = Program( H(qubit0) )
creg_index = 0

perfect_coin.measure(qubit0,creg_index)
cregs_to_check = [0]
trials = 5

print(qvm.run(perfect_coin, cregs_to_check, trials))

[[0], [0], [1], [1], [0]]


The only two lines of code we've neglected to examine thus far are:

---

cregs_to_check = [0]

trials = 5

---

which reappered in our .run() function:

---

qvm.run(perfect_coin, cregs_to_check, trials)

---

As mentioned before, the classical register is where we store our measurement results, and it is a string of bits, stored in an array, ex: [1,0,1,1,0,0,1,...].  Thus, ' cregs_to_check = [0] ' is simply defining which classical register indices we would like to check after we run the program.  In this case, we are only asking to check qubit 0.

If we want to check other indices, we can, although nothing will be there:

In [43]:
qubit0 = 0
perfect_coin = Program( H(qubit0) )
creg_index = 0

perfect_coin.measure(qubit0,creg_index)
cregs_to_check = [0,1,2,3]
trials = 5

print(qvm.run(perfect_coin, cregs_to_check, trials))

[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0]]


As expected, if we look at the classical register's indexs 1-3, we always get 0.  By default the entire classical register is all 0's.  If we wanted to, we could  choose to store our measurement result somewhere other than the same index as the qubit.  For example, let's store the measurement result of qubit 0 in say, index 2:

In [44]:
qubit0 = 0
perfect_coin = Program( H(qubit0) )
creg_index = 2

perfect_coin.measure(qubit0,creg_index)
cregs_to_check = [0,1,2,3]
trials = 5

print(qvm.run(perfect_coin, cregs_to_check, trials))

[[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


It's nice to have freedom in choosing where to store our measurement results, so long as we don't forget where we put them!

And lastly, in case you haven't figure it out by now, the 'trials' variable dictates how many times we would like to run the program.  But to be clear, we are specifying how many times we run the $\textit{whole}$ program, not just the measurement.  A measurement is $\textbf{permanent}$ (okay, I'll stop), so in order to run multiple measurements, we have to prepare the system from the beginning each time.

And that's it! We have successuly created a perfect coin flipping algorithm using 1 qubit.  After we measure the system, we can record the result and use it as we please.  

## Fancy Coin Algorithm
---

Let's use the code we just wrote, and clean it up a bit.  Specifically, let's create a custom function that performs the perfect coin flip and returns the result to us:

In [98]:
def Quantum_Coin_Flip():
    '''
    Uses a quantum system to simulate a perfect coin flip
    Returns a string either 'Heads' or 'Tails'
    '''
    perfect_coin = Program( H(0) ).measure(0,0)
    result = qvm.run(perfect_coin,[0],1)[0][0]
    if( result == 0 ):
        outcome = 'Heads'
    if( result == 1 ):
        outcome = 'Tails'
    return outcome
            
    
result = Quantum_Coin_Flip()   
print(result)   

Tails


Our Quantum_Coin_Flip works as intended - returning a string either 'Heads' or 'Tails'.  

Let's now take a look at how we condensed our code from the previous section, and extracted our measurement result for later use.

---

perfect_coin = Program( H(qubit0) ).measure(0,0)

---

This line of code condensed the initialization and measurement entries of our program down to one line.  Before, we were always doing:


---

perfect_coin = Program( )

perfect_coin.measure( )

---

This is totally fine, and for illustrative purposes it is very helpful in understanding the flow of the code.  Perpare it first... $\textit{check}$, then add a measurement to the program... $\textit{check}$.  But if we want to minimize our code, we can apply the .measure() function directly after Program().  So long as we have no intention of adding any more gates before the measurement, this is fine.

Next, let's take a look at how we got our measurement result out of the classical register.  To do this, let's first see the way in which the classical register's information comes back to us:

In [99]:
perfect_coin = Program( H(0) ).measure(0,0)
creg = qvm.run(perfect_coin,[0],1)

print('   creg: ',creg)

   creg:  [[0]]


Our classical register comes out of the .run() function as shown above.  This is an array, nested in another array.  The way to get these arrays and values out is as follows:

In [85]:
example_array = [[0]]
print('      example: ',example_array)
print('   example[0]: ',example_array[0])
print('example[0][0]: ',example_array[0][0])

      example:  [[0]]
   example[0]:  [0]
example[0][0]:  0


Thus, we can extract our measurement result by using:

---

qvm.run(perfect_coin,[0],1)[0][0]

---

To summarize, this line of code runs the program 'perfect_coin' exactly 1 time, and stores the measurement result to the classical register, in the location [0].  The function then resturns this classical register in the form of an array [[0,1,1,0,1,...]], which we then extract using [0][0] -- the first 0 grabs the inner array, while the second 0 grabs the 0$^{th}$ entry, which is where we chose to store our measurement result.

Let's see our perfect coin function in action!  The code below is a silly example, whereby we use the result of the measurement to determine a gambling bet:

In [97]:
import numpy as np
'''
This code simulates a gambling game between Alice and Bob.  
Alice has bet on more tosses of 'heads', while Bob has bet on 'Tails'
The loser must clean the other's quantum computer for a week
'''

Alice = 0
Bob = 0

coin_tosses = 100
for t in np.arange(coin_tosses):
    toss = Quantum_Coin_Flip()
    if(toss=='Heads'):
        Alice += 1
    if(toss=='Tails'):
        Bob   += 1
        
if(Alice > Bob):
    print('Looks like Bob has some cleaning to do!')
if(Alice < Bob):
    print('Tough luck Alice, Tails never Fails!')
if(Alice == Bob):
    print('Stupid quantum coin... Im going home')
print('  ')
print('Final Score -- Alice: ',Alice,' Bob: ',Bob)

Tough luck Alice, Tails never Fails!
  
Final Score -- Alice:  46  Bob:  54


I hope you enjoyed this lesson, and I encourage you to take a look at my other .ipynb tutorials!

link: https://github.com/dkoch92/Quantum-Algorithm-Tutorials